<p align = center>
    <img src="1_Overview.PNG" width=800 height=500 align=center />
<p/>

## Setup

In [1]:
#General
import numpy as np
import scipy
import scipy.linalg
import math

#Molecule Drivers
import pyscf
from pyscf import gto, scf, mcscf

##QISKIT
#Molecule, Fermioninc Operator, Problem Definition
from qiskit_nature.drivers import Molecule
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

#Properties
from qiskit_nature.properties import Property, GroupedProperty
from qiskit_nature.properties.second_quantization.electronic import (
    ElectronicEnergy,
    ElectronicDipoleMoment,
    ParticleNumber,
    AngularMomentum,
    Magnetization,
)
from qiskit_nature.properties.second_quantization.electronic.integrals import (
    ElectronicIntegrals,
    OneBodyElectronicIntegrals,
    TwoBodyElectronicIntegrals,
    IntegralProperty,
)
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis

#Qubit Operator
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization import QubitConverter

#Circuits, Simulators, Algorithms, Solvers and VQE
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, PauliTwoDesign
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver

##OPENFERMION
from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

##DWAVE
from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler()

##UTILITIES
from helper_functions import *
from XBK_method import *
from QCC_method import *


In [2]:
geometry = [['H', [  0.0,    0.0,    0.0]],
            ['H', [  0.0,    0.0,   0.739]]]

# geometry = [['H', [0.0, 0.0, 0.0]], ['H', [1.1, 0.0, 0.0]], ['H', [0.55, 0.9526279441628825, 0]]]

molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule=molecule, 
                                           basis='sto6g', 
                                           driver_type=ElectronicStructureDriverType.PYSCF)

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
fermionic_op = second_q_op[0]
of_fermionic_op = qiskit2of_fermionicOp(fermionic_op)
q_fermionic_op = of2qiskit_fermionicOp(of_fermionic_op)

of_fermionic_op = qiskit2of_fermionicOp(fermionic_op)

# es_problem.second_q_ops()[0] = q_fermionic_op
print(fermionic_op.to_list(), of_fermionic_op, q_fermionic_op.to_list())
# print(fermionic_op.to_list())
# print(q_fermionic_op.to_list())

[('+_0 -_1 +_2 -_3', (0.18144182452847726+0j)), ('+_0 -_1 -_2 +_3', (-0.1814418245284773+0j)), ('-_0 +_1 +_2 -_3', (-0.1814418245284773+0j)), ('-_0 +_1 -_2 +_3', (0.18144182452847724+0j)), ('+_3 -_3', (-0.47869740107940717+0j)), ('+_2 -_2', (-1.258194777661488+0j)), ('+_2 -_2 +_3 -_3', (0.4831168317179678+0j)), ('+_1 -_1', (-0.47869740107940717+0j)), ('+_1 -_1 +_3 -_3', (0.6994220820957332+0j)), ('+_1 -_1 +_2 -_2', (0.6645586562464451+0j)), ('+_0 -_0', (-1.258194777661488+0j)), ('+_0 -_0 +_3 -_3', (0.6645586562464451+0j)), ('+_0 -_0 +_2 -_2', (0.6748905616579757+0j)), ('+_0 -_0 +_1 -_1', (0.4831168317179678+0j))] (0.18144182452847724+0j) [0 1^ 2 3^] +
(-0.1814418245284773+0j) [0 1^ 2^ 3] +
(-1.258194777661488+0j) [0^ 0] +
(0.4831168317179678+0j) [0^ 0 1^ 1] +
(0.6748905616579757+0j) [0^ 0 2^ 2] +
(0.6645586562464451+0j) [0^ 0 3^ 3] +
(-0.1814418245284773+0j) [0^ 1 2 3^] +
(0.18144182452847726+0j) [0^ 1 2^ 3] +
(-0.47869740107940717+0j) [1^ 1] +
(0.6645586562464451+0j) [1^ 1 2^ 2] +
(0.

In [7]:
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(q_fermionic_op)
# print(qubit_op[4].to_spmatrix())
# qubit_matrix = qubit_op.to_matrix()
# qubit_op_spmatrix = qubit_op.to_spmatrix()

properties = driver.run()
print(properties)
electronic_energy = properties.get_property(ElectronicEnergy)
particle_number = properties.get_property(ParticleNumber)
num_electron = sum(particle_number.num_particles)
num_MO = num_electron
num_SO = particle_number.num_spin_orbitals
num_qubits = particle_number.num_spin_orbitals

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 2.0.1
		Config:
			atom=H 0.0 0.0 0.0;H 0.0 0.0 0.739
			unit=Angstrom
			charge=0
			spin=0
			basis=sto6g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.5487577448772091
		[0, 1] = -1.2133644630859333
		[1, 0] = 0.548757744877208
		[1, 1] = 1.2133644630859337
		Beta coefficients:
		[0, 0] = 0.5487577448772091
		[0, 1] = -1.2133644630859333
		[1, 0] = 0.548757744877208
		[1, 1] = 1.2133644630859337
	ParticleNumber:
		4 SOs
		1 alpha electrons
			orbital occupation: [1. 0.]
		1 beta electrons
			orbital occupation: [1. 0.]
	ElectronicEnergy
		(AO) 1-Body Terms:
			Alpha
			<(2, 2) matrix with 4 non-zero entries>
			[0, 0] = -1.125830094586104
			[0, 1] = -0.9632570528121425
			[1, 0] = -0.9632570528121425
			[1, 1] = -1.1258300945861035
			Beta
			<(2, 2) m

In [8]:
init_state = HartreeFock(num_SO, particle_number.num_particles, qubit_converter)
# init_state.draw()

# ansatz = UCCSD(qubit_converter, particle_number.num_particles, num_SO,vinitial_state = init_state)
ansatz = TwoLocal(rotation_blocks = ['h', 'rx', 'ry'], entanglement_blocks = 'cz', entanglement='linear', reps=3)
# ansatz.decompose().draw()

numpy_solver = NumPyMinimumEigensolver()
numpy_ground_state_solver = GroundStateEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_ground_state_solver.solve(es_problem)

exact_energy = numpy_results.computed_energies[0]
# print(f"Exact electronic energy: {exact_energy:.6f} Hartree\n")
# print(numpy_results)

backend = StatevectorSimulator()
optimizer = COBYLA()

error_threshold = 10 # mHartree

np.random.seed(5)  # fix seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

# for live plotting
# pp = ProgressPlot(plot_names=['Energy'],
#                   line_names=['Runtime VQE', f'Target + {error_threshold}mH', 'Target']) 
intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    # pp.update([[energy, exact_energy+error_threshold/1000, exact_energy]])

vqe = VQEUCCFactory(backend)
vqe_ground_state_solver = GroundStateEigensolver(qubit_converter, vqe)
vqe_results = vqe_ground_state_solver.solve(es_problem)

print(vqe_results)

error = (vqe_results.computed_energies[0] - exact_energy) * 1000 # mHartree
print(f'Error is: {error:.3f} mHartree')

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.862022752234
  - computed part:      -1.862022752234
~ Nuclear repulsion energy (Hartree): 0.716072003951
> Total ground state energy (Hartree): -1.145950748283
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.39650761]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.39650769]
    - computed part:      [0.0  0.0  1.39650769]
  > Dipole moment (a.u.): [0.0  0.0  -0.00000008]  Total: 0.00000008
                 (debye): [0.0  0.0  -0.00000019]  Total: 0.00000019
 
Error is: 0.000 mHartree


In [9]:
# #create molecule
# charge = 0
# multiplicity = 1
# basis = 'sto-6g'

# geometry = [['H', (  0.0,    0.0,    0.0)],
#             ['H', (  0.0,    0.0,   0.739)]]
# print(geometry)    
# molecule = MolecularData(
#     geometry=geometry,
#     basis=basis,
#     multiplicity=multiplicity,
#     charge=charge
# )

# driver = run_pyscf(molecule, run_scf=True)

# #convert to fermionic Hamiltonian
# molecular_H = molecule.get_molecular_hamiltonian()

# # if molecular_H[()] == None:
# #     molecular_H[()] = 0
# fermionic_H = get_fermion_operator(molecular_H)

# q_fermionic_op = of2qiskit_fermionicOp(fermionic_H)
# print(q_fermionic_op.to_list())

qubit_H = jordan_wigner(of_fermionic_op)
m = count_qubits(qubit_H)

from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

### XBK method ###

#set r value
r = 4

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(qubit_H, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

#run XBK method
XBK_energy, ground_state = XBK(qubit_Hs, qubit_Cs, r, sampler, starting_lam=0, num_samples=1000, strength=1e3, verbose=False)

print(XBK_energy)
print(ground_state) #ground state in rm-qubit space


-1.84149899365148
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]


# TEQUILA!

## Setup

In [66]:
#Execute the following in the terminal to set up conda

# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
# MINICONDA_PREFIX=/usr/local
# wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
# chmod +x $MINICONDA_INSTALLER_SCRIPT
# ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [75]:
!pip install tequila-basic

#https://pubs.acs.org/doi/10.1021/acs.jctc.7b00174 <-- whats that?
# !conda install -c conda-forge psi4 -c psi4 --yes

# !python -m pip install --user openfermion
# !python -m pip install --user openfermionpyscf

  Using cached https://files.pythonhosted.org/packages/55/de/bac1892ca411db3934fc62f49d4c2bebe1bf9759bd3584141ed4f6adda98/tequila_basic-1.6.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/60/18207ef16aa96ad4491da3e22e93b7eeb64d6a19b22f2d5e04a480aca6e7/cmake-3.21.4-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/72/6c/28e99141edfa6b22126e5d873dd62ce6b9987aebf66d33a98bfe42242f41/openfermion-1.0.0-py3-none-any.whl
  Could not find a version that satisfies the requirement jaxlib (from tequila-basic) (from versions: )
No matching distribution found for jaxlib (from tequila-basic)


## Code

In [77]:
import numpy as np
import tequila as tq
from utility import *

ModuleNotFoundError: No module named 'tequila'

# D-Wave

## Setup

In [78]:
#!curl  -H "X-Auth-Token: YourAPIkey" -X DELETE https://cloud.dwavesys.com/sapi/problems/7211d196-cfc6-4630-84d0-f42ae5d55f0f

#https://www.youtube.com/watch?v=bErs0dxC1aY&list=PLPvKnT7dgEsuhec_yW9oxcZ6gg9SfRFFV
#https://docs.ocean.dwavesys.com/en/latest/overview/install.html#installoceansoftware
#!python -m venv ocean
#!\Scripts\activate

# !pip install dwave-ocean-sdk
# !pip install dwave-qiskit-plugin
# !pip install requests --upgrade --yes
# !dwave setup #API endpoint URL: https://cloud.dwavesys.com/sapi/

#RESTART RUNTIME

## Potentially Useful Junk

In [ ]:
# !pip install pyscf
# !pip install qiskit-nature[pyscf]
# !pip install qiskit

# try:
#     import openfermion
# except ImportError:
#     !pip install -q git+https://github.com/quantumlib/OpenFermion.git@master#egg=openfermion

#with qiskit

#without qiskit (maybe more flexible when working between packages)
# molecule = "H .0 .0 .0; H .0 .0 0.739"
# driver = PySCFDriver(atom=molecule)
# qmolecule = driver.run()
# electronic_energy = qmolecule.get_property(ElectronicEnergy)
# one_body_ints = electronic_energy.get_electronic_integral(ElectronicBasis.MO, 1)
# h1 = qmolecule.one_body_integrals.diagonal()
# h2 = qmolecule.two_body_integrals
# print(h1, h2)

#https://quantumcomputing.stackexchange.com/questions/13803/what-is-the-convention-of-indices-for-the-one-and-two-body-integrals-in-qiskit

# ##some qiskit properties
# properties = driver.run()
# print(properties)
# electronic_energy = properties.get_property(ElectronicEnergy)
# particle_number = properties.get_property(ParticleNumber)
# num_electron = sum(particle_number.num_particles)
# num_MO = num_electron
# num_SO = particle_number.num_spin_orbitals
# num_qubits = particle_number.num_spin_orbitals

# some_properties = {
#     'electronic_energy': electronic_energy.reference_energy,
#     'electrons': num_electron,
#     'MOs': num_MO,
#     'SOs': num_SO,
#     'qubits': num_qubits
# }
# # print(some_properties)

##iterate soarse matrix
# import itertools
# import random

# def using_tocoo_zip(x):
#     cx = x.tocoo()    
#     for i,j,v in zip(cx.row, cx.col, cx.data):
#         print((i,j),(np.real(v), np.imag(v)))

# using_tocoo_zip(qubit_op_spmatrix)